In [ ]:
import os
import math
import time
from collections import deque

import numpy as np
import matplotlib.pyplot as plt

from shapely.geometry import Polygon

In [2]:
class Point:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        self.adj_edges = []

    def __call__(self) -> tuple:
        return (self.x, self.y, self.z)

    def __str__(self) -> str:
        return str((self.x, self.y, self.z))

    def __eq__(self, other):
        return self.__dict__ == other.__dict__

    def add_adj_edge(self, edge):
        self.adj_edges.append(edge)

    def point_is_close(self, p):
        if math.isclose(self.x,p.x) and math.isclose(self.y,p.y) and math.isclose(self.z, p.z):
            return True
        return False
            
        
class Edge:
    def __init__(self, p1, p2, visits=0):
        if p1.point_is_close(p2):
            raise Exception("Edge must have two different points. Points given: {} and {}.".format(p1, p2))
        self.p1 = p1
        self.p2 = p2
        self.visits = visits
        p1.add_adj_edge(self)
        p2.add_adj_edge(self)

    def __str__(self):
        return "{} {}".format(str(self.p1), str(self.p2))

    def __eq__(self, other):
        if self.p1 == other.p1 or self.p1 == other.p2:
            if self.p2 == other.p1 or self.p2 == other.p2:
                return True
        return False


class Face():
    def __init__(self, p1, p2, p3):
        # super().__init__(self)
        self.p1 = p1
        self.p2 = p2
        self.p3 = p3

        self.e1 = Edge(p1, p2)
        self.e2 = Edge(p2, p3)
        self.e3 = Edge(p3, p1)

        self.spoly = Polygon([[p1.x,p1.y,p1.z],[p2.x,p2.y,p2.z],[p3.x,p3.y,p3.z]])
    def __str__(self):
        return "{} {} {}".format(str(self.p1), str(self.p2), str(self.p3))

    def __eq__(self, other):
        if self.p1.point_is_close(other.p1) or self.p1.point_is_close(other.p2) or self.p1.point_is_close(other.p3):
            if self.p2.point_is_close(other.p1) or self.p2.point_is_close(other.p2) or self.p2.point_is_close(other.p3):
                if self.p3.point_is_close(other.p1) or self.p3.point_is_close(other.p2) or self.p3.point_is_close(other.p3):
                    return True
        return False

In [3]:
def read_vertices_from_obj(objPath):
    points = []
    with open(objPath, 'r') as f:
        for line in f:
            try:
                line = line.replace('v', '')
            except:
                #                 print("Problema na linha")
                continue
            line_ = line.split()
            try:
                points.append(
                    Point(float(line_[0]), float(line_[1]), float(line_[2])))
            except:
                #                 print("Problema no ponto")
                continue
    return points
  
        
def read_faces_from_obj(objPath):
    vertices = read_vertices_from_obj(objPath)
    faces = []
    with open(objPath, 'r') as f:
        for line in f:
            try:
                line = line.replace('f', '')
            except:
                continue
            line_ = line.split()
            try:
                new_face = Face(vertices[int(line_[0])-1], vertices[int(line_[1])-1], vertices[int(line_[2])-1])
                faces.append(new_face)
            except Exception as e:
                # print(e)
                continue
    return faces


def write_faces(F, P, filename=False):
    i1 = 0
    i2 = 0
    i3 = 0
    if not filename:
        filename = os.path.join(os.getcwd(), 'obj', 'faces.obj')
    with open(filename, 'w') as f:
        for point in P:
            f.write("v {} {} {}".format(point.x, point.y, point.z))
            f.write('\n')
        f.write('\n')
        for i in range(len(F)):
            for j in range(len(P)):
                if F[i].p1.x == P[j].x and F[i].p1.y == P[j].y and F[i].p1.z == P[j].z:
                    i1 = j
                if F[i].p2.x == P[j].x and F[i].p2.y == P[j].y and F[i].p2.z == P[j].z:
                    i2 = j
                if F[i].p3.x == P[j].x and F[i].p3.y == P[j].y and F[i].p3.z == P[j].z:
                    i3 = j
            f.writelines("f {} {} {}".format(i1+1, i2+1, i3+1))
            f.write('\n')

In [4]:
def cross(a, b):
    return Point(a.y*b.z-a.z*b.y, a.z*b.x-a.x*b.z, a.x*b.y-a.y*b.x)

def dot(a, b):
    return (a.x*b.x)+(a.y*b.y)+(a.z*b.z)

def norma(a):
    return math.sqrt(a.x*a.x + a.y*a.y + a.z*a.z)

def sub(a, b):
    return Point(a.x-b.x, a.y-b.y, a.z-b.z)

def normal(face):
    v1 = sub(face.p2, face.p1)    
    v2 = sub(face.p3, face.p1)
    return cross(v1,v2)

def distance_point_to_plane(p, n, f):
    a = n.x
    b = n.y
    c = n.z
    d = a*f.p1.x + b*f.p1.y + c*f.p1.z
    dist = (a*p.x+b*p.y+c*p.z+d)/math.sqrt(a**2+b**2+c**2)
    dist = abs(dist)
    return dist

In [5]:
def check_intersection_faces(point, face, faces):
    """Checks if the faces created with point intersect 
    a set of known faces.

    Args:
        point (Point): Point to verify
        face (Face): Base face
        faces (list): List of known faces
    """
    new_faces = []
    f1 = Face(face.e1.p1, point, face.e1.p2)
    f2 = Face(face.e2.p1, point, face.e2.p2)
    f3 = Face(face.e3.p1, point, face.e3.p2)
    for f in faces:
        if f.spoly.crosses(f1.spoly):
            return True
        if f.spoly.crosses(f2.spoly):
            return True
        if f.spoly.crosses(f3.spoly):
            return True
    return False

In [14]:
def check_if_face_already_exists(face_candidate, faces):
    """Returns True if face_candidate is already in faces

    Args:
        face_candidate (Face): Face candidate to enter the frontier
        faces (list): List of faces already in the frontier
    """
#     print("Face candidata:",face_candidate)
    for face in faces:
#         print(face)
        if face_candidate == face:
            return True
    return False

In [25]:
def avanco_de_fronteira_3d(hull, points):
    faces = hull.copy()
    faces_obtidas = hull.copy()
    print(len(hull), len(points))

    max_dist = -float("inf")
    debug = 0
    for f in faces:
        candidate = None
        max_dist = -float("inf")
        distances = []
        normal_1 = normal(f)
        for p in points:
            if p.point_is_close(f.p1) or p.point_is_close(f.p2) or p.point_is_close(f.p3):
                continue
            dist = distance_point_to_plane(p, normal_1, f)
            distances.append(dist)
            
        points = np.array(points)
        indexes = np.argsort(distances)
        points = points[indexes.tolist()]
        points = points.tolist()
        for i in range(len(points)):
            
            if points[i].point_is_close(f.p1) or points[i].point_is_close(f.p2) or points[i].point_is_close(f.p3):
                continue
            
            candidate = None
            dist = distances[i]
            if dist > max_dist:
                if not check_intersection_faces(points[i], f, faces_obtidas):
                    max_dist = dist
                    candidate = points[i]
            if candidate is not None:
                f1 = Face(f.e1.p1, candidate, f.e1.p2)
                f2 = Face(f.e2.p1, candidate, f.e2.p2)
                f3 = Face(f.e3.p1, candidate, f.e3.p2)


                if not check_if_face_already_exists(f1, faces) and not f1 == f:
                    if not check_if_face_already_exists(f2, faces) and not f2 == f:
                        if not check_if_face_already_exists(f3, faces) and not f3 == f:
                            faces.append(f1)
                            faces.append(f2)
                            faces.append(f3)
                if not check_if_face_already_exists(f1, faces_obtidas) and not f1 == f:
                    if not check_if_face_already_exists(f2, faces_obtidas) and not f2 == f:
                        if not check_if_face_already_exists(f3, faces_obtidas) and not f3 == f:
                            faces_obtidas.append(f1)
                            faces_obtidas.append(f2)
                            faces_obtidas.append(f3)
                break
    return faces_obtidas

In [24]:
obj_file = 'obj/hullControle01.obj'
faces = read_faces_from_obj(obj_file)
vertices = read_vertices_from_obj(obj_file)
res = avanco_de_fronteira_3d(faces, vertices)
print(len(res))
write_faces(res, vertices, 'obj/tetra_controle01.obj')

692 14526
2003
